In [26]:
import pandas as pd
import matplotlib.pyplot as plt


In [48]:

df = pd.read_csv("Loan 2.csv", encoding = "ISO-8859-1")

In [49]:
# Display the first few rows of the merged dataframe
print(df.head())

  Customer ID               Name Gender  Age  Income (USD) Income Stability  \
0     C-36995   Frederica Shealy      F   56       1933.05              Low   
1     C-33999  America Calderone      M   32       4952.91              Low   
2      C-3770      Rosetta Verne      F   65        988.19             High   
3     C-26480         Zoe Chitty      F   65           NaN             High   
4     C-23459       Afton Venema      F   31       2614.77              Low   

  Profession     Type of Employment    Location  Loan Amount Request (USD)  \
0    Working            Sales staff  Semi-Urban                   72809.58   
1    Working                    NaN  Semi-Urban                   46837.47   
2  Pensioner                    NaN  Semi-Urban                   45593.04   
3  Pensioner                    NaN       Rural                   80057.92   
4    Working  High skill tech staff  Semi-Urban                  113858.89   

   Current Loan Expenses (USD) Expense Type 1 Expense Ty

 print(merged_df.isnull().sum())

In [51]:
print(df.isnull().sum())

Customer ID                       0
Name                              0
Gender                           53
Age                               0
Income (USD)                   4576
Income Stability               1683
Profession                        0
Type of Employment             7270
Location                          0
Loan Amount Request (USD)         0
Current Loan Expenses (USD)     172
Expense Type 1                    0
Expense Type 2                    0
Dependents                     2493
Credit Score                   1703
No. of Defaults                   0
Has Active Credit Card         1566
Property ID                       0
Property Age                   4850
Property Type                     0
Property Location               356
Co-Applicant                      0
Property Price                    0
Loan Sanction Amount (USD)      340
dtype: int64


In [21]:
# # Drop columns with more than 60% missing values
# threshold = len(merged_df) * 0.6
# merged_df = merged_df.dropna(thresh=threshold, axis=1)

# print("Remaining columns after dropping:\n", merged_df.isnull().sum())

In [52]:
# Calculate the mode of the Gender
gender_mode = df['Gender'].mode()[0]
print(f"Mode of Gender column: {gender_mode}")

Mode of Gender column: M


In [53]:
# Fill missing values with the mode
df['Gender'].fillna(gender_mode, inplace=True)

In [54]:
print(df['Gender'].isnull().sum())

0


In [55]:
# Calculate the mean of the Income
income_mean = df['Income (USD)'].mean()
print(f"Mean of Income (USD) column: {income_mean}")

Mean of Income (USD) column: 2630.574417479547


In [56]:
df['Income (USD)'].fillna(income_mean, inplace=True)

In [58]:
conditions=[
    (df['Income Stability'].notnull()),
    (df['Income Stability'].isnull())&(df['Profession']=='Commercial associate'),#Low
    (df['Income Stability'].isnull())&(df['Profession']=='Working'),#Low
    (df['Income Stability'].isnull())&(df['Profession']=='Pensioner'),#High
    (df['Income Stability'].isnull())&(df['Profession']=='State servant'),#Low
    (df['Income Stability'].isnull())&(df['Profession']=='Unemployed'),#Low
    (df['Income Stability'].isnull())&(df['Profession']=='Maternity leave'),#Low
    (df['Income Stability'].isnull())&(df['Profession']=='Student'),#Low
    (df['Income Stability'].isnull())&(df['Profession']=='Businessman')#Low    
]

values=[
    df['Income Stability'],
    'Low',
    'Low',
    'High',
    'Low',
    'Low',
    'Low',
    'Low',
    'High'
]


In [59]:
df['Income Stability'] = np.select(conditions, values, default='Unknown')

In [60]:
df['Income Stability'].isnull().sum()

0

In [61]:
# Calculate the mode of the Type of Employment column
employment_mode = df['Type of Employment'].mode()[0]
print(f"Mode of Type of Employment column: {employment_mode}")

Mode of Type of Employment column: Laborers


In [62]:

# Fill missing values with the mode
df['Type of Employment'].fillna(employment_mode, inplace=True)

In [63]:
# Calculate the median of the Current Loan Expenses (USD) column
loan_expenses_median = df['Current Loan Expenses (USD)'].median()
print(f"Median of Current Loan Expenses (USD) column: {loan_expenses_median}")

Median of Current Loan Expenses (USD) column: 375.205


In [64]:
df['Current Loan Expenses (USD)'].fillna(loan_expenses_median, inplace=True)

In [65]:
# Fill missing values in predictors with their median
df['Income (USD)'].fillna(df['Income (USD)'].median(), inplace=True)
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Current Loan Expenses (USD)'].fillna(df['Current Loan Expenses (USD)'].median(), inplace=True)

# Check for remaining missing values
print(df[['Income (USD)', 'Age', 'Current Loan Expenses (USD)']].isnull().sum())

Income (USD)                   0
Age                            0
Current Loan Expenses (USD)    0
dtype: int64


In [66]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Separate the dataset into training data \
train_data = df[df['Loan Sanction Amount (USD)'].notnull()]
predict_data = df[df['Loan Sanction Amount (USD)'].isnull()]

# Define predictor variables (X) and target variable (y)
X_train = train_data[['Income (USD)', 'Age', 'Current Loan Expenses (USD)']]
y_train = train_data['Loan Sanction Amount (USD)']

# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict the missing values
X_predict = predict_data[['Income (USD)', 'Age', 'Current Loan Expenses (USD)']]
predicted_values = model.predict(X_predict)

# Fill the missing values
df.loc[df['Loan Sanction Amount (USD)'].isnull(), 'Loan Sanction Amount (USD)'] = predicted_values

In [68]:
from sklearn.linear_model import LinearRegression

# Separate the dataset into training data (without nulls in target) and data to predict (with nulls in target)
train_data = df[df['Property Age'].notnull()]
predict_data = df[df['Property Age'].isnull()]

# Define predictor variables (X) and target variable (y)
X_train = train_data[['Income (USD)', 'Age', 'Current Loan Expenses (USD)']]
y_train = train_data['Property Age']

# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict the missing values
X_predict = predict_data[['Income (USD)', 'Age', 'Current Loan Expenses (USD)']]
predicted_values = model.predict(X_predict)

# Fill the missing values
df.loc[df['Property Age'].isnull(), 'Property Age'] = predicted_values

In [72]:
print(df.isnull().sum())

Customer ID                       0
Name                              0
Gender                            0
Age                               0
Income (USD)                      0
Income Stability                  0
Profession                        0
Type of Employment                0
Location                          0
Loan Amount Request (USD)         0
Current Loan Expenses (USD)       0
Expense Type 1                    0
Expense Type 2                    0
Dependents                     2493
Credit Score                   1703
No. of Defaults                   0
Has Active Credit Card            0
Property ID                       0
Property Age                      0
Property Type                     0
Property Location               356
Co-Applicant                      0
Property Price                    0
Loan Sanction Amount (USD)        0
dtype: int64


In [76]:
# Calculate the mode of the Gender
dependents_mode = df['Dependents'].mode()[0]
print(f"Mode of dependents column: {dependents_mode}")

Mode of dependents column: 2.0


In [81]:
# Fill missing values with the mode
df['Dependents'].fillna(dependents_mode, inplace=True)

In [86]:
# Calculate the median of credit_score
credit_median = df['Credit Score'].median()
print(f"median of credit score column: {credit_median}")

median of credit score column: 739.82


In [87]:
# Fill missing values with the median
df['Credit Score'].fillna(credit_median, inplace=True)

In [88]:
print(df.isnull().sum())


Customer ID                      0
Name                             0
Gender                           0
Age                              0
Income (USD)                     0
Income Stability                 0
Profession                       0
Type of Employment               0
Location                         0
Loan Amount Request (USD)        0
Current Loan Expenses (USD)      0
Expense Type 1                   0
Expense Type 2                   0
Dependents                       0
Credit Score                     0
No. of Defaults                  0
Has Active Credit Card           0
Property ID                      0
Property Age                     0
Property Type                    0
Property Location              356
Co-Applicant                     0
Property Price                   0
Loan Sanction Amount (USD)       0
dtype: int64


In [89]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = df['Income (USD)'].quantile(0.25)
Q3 = df['Income (USD)'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['Income (USD)'] < lower_bound) | (df['Income (USD)'] > upper_bound)]

print(f"Outliers:\n{outliers}")

Outliers:
      Customer ID               Name Gender  Age  Income (USD)  \
1         C-33999  America Calderone      M   32       4952.91   
27        C-44863     Gerard Forsman      F   18       5350.94   
36        C-21055    Travis Priester      F   61       5302.28   
39        C-24451     Margareta Wind      M   18       7885.56   
41        C-48527       Mana Chattin      F   54       5330.72   
...           ...                ...    ...  ...           ...   
29892     C-22791       Ranee Faunce      M   18       5074.06   
29904      C-5271     Londa Eisenman      M   60       6685.40   
29913     C-40023      Zoraida Verde      M   29       4665.26   
29986     C-36230       Krystin Weir      M   32       8308.27   
29995     C-43723  Angelyn Clevenger      M   38       4969.41   

      Income Stability            Profession Type of Employment    Location  \
1                  Low               Working           Laborers  Semi-Urban   
27                 Low  Commercial asso

In [90]:
# Replacing outliers with the median value of the column
median_value = df['Income (USD)'].median()
df['Income (USD)'] = df['Income (USD)'].apply(lambda x: median_value if (x < lower_bound or x > upper_bound) else x)

print(f"Data with Replaced Outliers:\n{df}")

Data with Replaced Outliers:
      Customer ID               Name Gender  Age  Income (USD)  \
0         C-36995   Frederica Shealy      F   56   1933.050000   
1         C-33999  America Calderone      M   32   2473.470000   
2          C-3770      Rosetta Verne      F   65    988.190000   
3         C-26480         Zoe Chitty      F   65   2630.574417   
4         C-23459       Afton Venema      F   31   2614.770000   
...           ...                ...    ...  ...           ...   
29995     C-43723  Angelyn Clevenger      M   38   2473.470000   
29996     C-32511       Silas Slaugh      M   20   1606.880000   
29997      C-5192       Carmelo Lone      F   49   2630.574417   
29998     C-12172      Carolann Osby      M   38   2417.710000   
29999     C-33003  Bridget Garibaldi      F   63   3068.240000   

      Income Stability            Profession     Type of Employment  \
0                  Low               Working            Sales staff   
1                  Low              

In [91]:
Q1 = df['Loan Sanction Amount (USD)'].quantile(0.25)
Q3 = df['Loan Sanction Amount (USD)'].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers_iqr = df[(df['Loan Sanction Amount (USD)'] < lower_bound) | (df['Loan Sanction Amount (USD)'] > upper_bound)]

print(f"Outliers using IQR:\n{outliers_iqr}")

Outliers using IQR:
      Customer ID                Name Gender  Age  Income (USD)  \
87        C-27965    Jacklyn Jiggetts      F   61   2630.574417   
215       C-16868    Earlie Fleischer      M   64   2473.470000   
247       C-34512        Melonie Pare      M   52   3019.220000   
267        C-7202  Sanford Laughridge      M   25   2630.574417   
305       C-26752          Guy Huling      M   62   2473.470000   
...           ...                 ...    ...  ...           ...   
29538      C-2791           Ned Nicks      M   32   3662.450000   
29615     C-21328   Princess Bettcher      M   18   3893.550000   
29658     C-40788       Lolita Adkins      F   53   2473.470000   
29904      C-5271      Londa Eisenman      M   60   2473.470000   
29918      C-9504           Alaina Au      F   64   3483.690000   

      Income Stability            Profession Type of Employment    Location  \
87                High             Pensioner           Laborers       Urban   
215              

In [92]:
# Replace outliers with the median value
median_value = df['Loan Sanction Amount (USD)'].median()
df['Loan Sanction Amount (USD)'] = df['Loan Sanction Amount (USD)'].apply(lambda x: median_value if (x < lower_bound or x > upper_bound) else x)

print(f"Data after replacing outliers with median:\n{df}")

Data after replacing outliers with median:
      Customer ID               Name Gender  Age  Income (USD)  \
0         C-36995   Frederica Shealy      F   56   1933.050000   
1         C-33999  America Calderone      M   32   2473.470000   
2          C-3770      Rosetta Verne      F   65    988.190000   
3         C-26480         Zoe Chitty      F   65   2630.574417   
4         C-23459       Afton Venema      F   31   2614.770000   
...           ...                ...    ...  ...           ...   
29995     C-43723  Angelyn Clevenger      M   38   2473.470000   
29996     C-32511       Silas Slaugh      M   20   1606.880000   
29997      C-5192       Carmelo Lone      F   49   2630.574417   
29998     C-12172      Carolann Osby      M   38   2417.710000   
29999     C-33003  Bridget Garibaldi      F   63   3068.240000   

      Income Stability            Profession     Type of Employment  \
0                  Low               Working            Sales staff   
1                  Low

In [93]:
Q1 = df['Loan Amount Request (USD)'].quantile(0.25)
Q3 = df['Loan Amount Request (USD)'].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers_iqr = df[(df['Loan Amount Request (USD)'] < lower_bound) | (df['Loan Amount Request (USD)'] > upper_bound)]



# Replace outliers with the median value
median_value = df['Loan Amount Request (USD)'].median()
df['Loan Amount Request (USD)'] = df['Loan Amount Request (USD)'].apply(lambda x: median_value if (x < lower_bound or x > upper_bound) else x)

print(f"Data after replacing outliers with median:\n{df}")



Data after replacing outliers with median:
      Customer ID               Name Gender  Age  Income (USD)  \
0         C-36995   Frederica Shealy      F   56   1933.050000   
1         C-33999  America Calderone      M   32   2473.470000   
2          C-3770      Rosetta Verne      F   65    988.190000   
3         C-26480         Zoe Chitty      F   65   2630.574417   
4         C-23459       Afton Venema      F   31   2614.770000   
...           ...                ...    ...  ...           ...   
29995     C-43723  Angelyn Clevenger      M   38   2473.470000   
29996     C-32511       Silas Slaugh      M   20   1606.880000   
29997      C-5192       Carmelo Lone      F   49   2630.574417   
29998     C-12172      Carolann Osby      M   38   2417.710000   
29999     C-33003  Bridget Garibaldi      F   63   3068.240000   

      Income Stability            Profession     Type of Employment  \
0                  Low               Working            Sales staff   
1                  Low

In [94]:
Q1 = df['Current Loan Expenses (USD)'].quantile(0.25)
Q3 = df['Current Loan Expenses (USD)'].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers_iqr = df[(df['Current Loan Expenses (USD)'] < lower_bound) | (df['Current Loan Expenses (USD)'] > upper_bound)]



# Replace outliers with the median value
median_value = df['Loan Amount Request (USD)'].median()
df['Current Loan Expenses (USD)'] = df['Current Loan Expenses (USD)'].apply(lambda x: median_value if (x < lower_bound or x > upper_bound) else x)

print(f"Data after replacing outliers with median:\n{df}")

Data after replacing outliers with median:
      Customer ID               Name Gender  Age  Income (USD)  \
0         C-36995   Frederica Shealy      F   56   1933.050000   
1         C-33999  America Calderone      M   32   2473.470000   
2          C-3770      Rosetta Verne      F   65    988.190000   
3         C-26480         Zoe Chitty      F   65   2630.574417   
4         C-23459       Afton Venema      F   31   2614.770000   
...           ...                ...    ...  ...           ...   
29995     C-43723  Angelyn Clevenger      M   38   2473.470000   
29996     C-32511       Silas Slaugh      M   20   1606.880000   
29997      C-5192       Carmelo Lone      F   49   2630.574417   
29998     C-12172      Carolann Osby      M   38   2417.710000   
29999     C-33003  Bridget Garibaldi      F   63   3068.240000   

      Income Stability            Profession     Type of Employment  \
0                  Low               Working            Sales staff   
1                  Low

In [97]:
Q1 = df['Property Price'].quantile(0.25)
Q3 = df['Property Price'].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers_iqr = df[(df['Property Price'] < lower_bound) | (df['Property Price'] > upper_bound)]



# Replace outliers with the median value
median_value = df['Property Price'].median()
df['DProperty Price'] = df['Property Price'].apply(lambda x: median_value if (x < lower_bound or x > upper_bound) else x)

print(f"Data after replacing outliers with median:\n{df}")

Data after replacing outliers with median:
      Customer ID               Name Gender  Age  Income (USD)  \
0         C-36995   Frederica Shealy      F   56   1933.050000   
1         C-33999  America Calderone      M   32   2473.470000   
2          C-3770      Rosetta Verne      F   65    988.190000   
3         C-26480         Zoe Chitty      F   65   2630.574417   
4         C-23459       Afton Venema      F   31   2614.770000   
...           ...                ...    ...  ...           ...   
29995     C-43723  Angelyn Clevenger      M   38   2473.470000   
29996     C-32511       Silas Slaugh      M   20   1606.880000   
29997      C-5192       Carmelo Lone      F   49   2630.574417   
29998     C-12172      Carolann Osby      M   38   2417.710000   
29999     C-33003  Bridget Garibaldi      F   63   3068.240000   

      Income Stability            Profession     Type of Employment  \
0                  Low               Working            Sales staff   
1                  Low

In [98]:
Q1 = df['Credit Score'].quantile(0.25)
Q3 = df['Credit Score'].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers_iqr = df[(df['Credit Score'] < lower_bound) | (df['Credit Score'] > upper_bound)]



# Replace outliers with the median value
median_value = df['Credit Score'].median()
df['Credit Score'] = df['Credit Score'].apply(lambda x: median_value if (x < lower_bound or x > upper_bound) else x)

print(f"Data after replacing outliers with median:\n{df}")

Data after replacing outliers with median:
      Customer ID               Name Gender  Age  Income (USD)  \
0         C-36995   Frederica Shealy      F   56   1933.050000   
1         C-33999  America Calderone      M   32   2473.470000   
2          C-3770      Rosetta Verne      F   65    988.190000   
3         C-26480         Zoe Chitty      F   65   2630.574417   
4         C-23459       Afton Venema      F   31   2614.770000   
...           ...                ...    ...  ...           ...   
29995     C-43723  Angelyn Clevenger      M   38   2473.470000   
29996     C-32511       Silas Slaugh      M   20   1606.880000   
29997      C-5192       Carmelo Lone      F   49   2630.574417   
29998     C-12172      Carolann Osby      M   38   2417.710000   
29999     C-33003  Bridget Garibaldi      F   63   3068.240000   

      Income Stability            Profession     Type of Employment  \
0                  Low               Working            Sales staff   
1                  Low